In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
# # tag_expression and code block for tagging parcels
# tag_expression = "createTag(!TAG!, '{}')"
# tag_codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

In [3]:
# inputs
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb\\Parcels_Weber_LIR_UTM12'

In [4]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "classes.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "classes.gdb")
    
scratch = os.path.join(outputs, "scratch.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch.gdb")
    
final = os.path.join(outputs, "final.gdb")
if not arcpy.Exists(final):
    arcpy.CreateFileGDB_management(outputs, "final.gdb")

In [5]:
# get address points without base address point
address_pts_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\AddressPoints.gdb\\AddressPoints_Weber','address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, scratch, '_00_address_pts_no_base')

In [6]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, scratch, '_01_parcels_for_modeling')

# recalc acreage 
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

# create the main layer
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr') 

In [7]:
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')

# add REMM fields
arcpy.AddField_management(parcels_for_modeling_layer, 'max_dua', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_far', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_height', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type1', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type2', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type3', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type4', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type5', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type6', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type7', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type8', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'agriculture', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'basebldg', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NoBuild', 'SHORT') # REMM doesn't use this
arcpy.AddField_management(parcels_for_modeling_layer, 'redev_friction', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'building_type_id', 'SHORT')

# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)

# calculate default for basebldg field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='basebldg', expression="0", expression_type="PYTHON3")

# calculate default for parcel type fields
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type1', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type2', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type3', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type4', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type5', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type6', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type7', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type8', expression="'f'", expression_type="PYTHON3")        
        
        
# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))

# initial parcels in modeling area:
 96977


## Categorize parcels within Owned Unit Groupings

In [8]:
###############
# Common Areas
###############

common_areas = r'.\Inputs\Weber_Owned_Unit_Groupings.gdb\Weber_Owned_Unit_Groupings'
common_areas_lyr = arcpy.MakeFeatureLayer_management(common_areas, 'common_areas_lyr') 

# open_space
query = """ TYPE_WFRC IN ('open_space') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_open_space = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02a_ca_open_space')

# vacant
query = """ TYPE_WFRC IN ('vacant') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_vacant = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02b_ca_vacant')


# commercial
query = """ TYPE_WFRC IN ('commercial') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_commercial = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02d_ca_commercial')

# pud
query = """ SUBTYPE_WFRC IN ('pud') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_pud = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02h_ca_pud')

# multi_family
query = """ TYPE_WFRC IN ('multi_family') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_multi_family = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02i_ca_multi_family')


In [9]:
###############
# PUD
###############

tag = "single_family"
tag2 = "pud"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_pud, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_pud_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_pud, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_pud
join_features = pud_centroids
output_features = os.path.join(scratch, "_03a_pud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(oug_sj, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2107 "single_family" parcels were selected.
94870 parcels remain...


In [10]:
###############
# multi family
###############

tag = "multi_family"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_multi_family, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_mf_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_multi_family, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_multi_family
join_features = mf_centroids
output_features = os.path.join(scratch, "_03b_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

8782 "multi_family" parcels were selected.
86088 parcels remain...


In [11]:
#####################
# commercial commons
#####################

tag = "commercial"

# parcels that are contained by industrial common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_commercial, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
commercial_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_commercial_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_commercial, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_commercial
join_features = commercial_centroids
output_features = os.path.join(scratch, "_03e_commercial_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_commercial_commons")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1094 "commercial" parcels were selected.
84994 parcels remain...


## Categorize parcels

In [12]:
###########
# churches
###########

query = (""" TAXEXEMPT_TYPE IN ('801-LDS Church','803-Baptist Church', '804-Buddhist Church', '805-Catholic Church', 
            '806-Other Denominations') OR PARCEL_ID IN ('240290002','240290003') """) 
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="8",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

490 "churches" parcels were selected.
84504 parcels remain...


In [13]:
###############
# agriculture
###############

query = """ PROP_CLASS IN ('811- FARM ANIMALS','812- PROD FARM') """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

404 "agriculture" parcels were selected.
84100 parcels remain...


In [14]:
###############
# education
###############

query = (""" TAXEXEMPT_TYPE IN ('310-Ogden City Schools','320-Weber County Schools','340-WSU') OR 
             PARCEL_ID IN ('130260001','130050091') """)
tag="education"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

302 "education" parcels were selected.
83798 parcels remain...


In [15]:
################
# single_family
################

query = (''' PROP_CLASS IN ('108- SFR on COMM Zone', '111- SNGL FAM RES', '502-COMM LAND with SFR', 
            '508- Both RES & COMM IMPRVD') OR PARCEL_ID IN ('161300020', '010600021','130340037') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calulate building field to "yes"

# these parcels are probably sf residential, but with no building
query = (''' PROP_CLASS IN ('121-OUT BLDGS', '957-RELATED PARCEL') OR PARCEL_ID IN ('130060033','130170033') ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

485 "single_family" parcels were selected.
83313 parcels remain...


In [16]:
################
# Duplex
################

tag="duplex"

query = (''' PROP_CLASS IN ('112- DUPLEX') ''')

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1483 "duplex" parcels were selected.
81830 parcels remain...


In [17]:
################
# Multi-Family
################

query = (''' PROP_CLASS IN ('113- 3-4 UNITS','114- 5-9 UNITS', '115- 10+ UNITS','300- MULTIPLE HOUSING') OR 
            PARCEL_ID IN ('010600021', '122210002','120840030', '120310022')''')
tag="multi_family2"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mf2_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_mf2_commons')

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mf2_commons
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family2")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


831 "multi_family2" parcels were selected.
80999 parcels remain...


In [18]:
#################
# mobile home parks
#################

tag = "multi_family"
tag2 = "mobile_home_park"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

query= (""" PROP_CLASS IN ('118- MOBILE HOME') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(mhp, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))




50 "multi_family" parcels were selected.
80949 parcels remain...


In [19]:
###############
# utilities
###############

query= (""" TAXEXEMPT_TYPE IN ('601-Mountain Fuel (Questar)','602-Utah Power and Light','603-US West Telephone',
            '604-All Other Utilities', '605-Weber Basin Water','607-Water Tank Parcel','609-Water & Sewer Districts', 
            '995-Taxable Water Companies') """)

tag="undevelopable"
tag2="utilities"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

502 "undevelopable" parcels were selected.
80447 parcels remain...


In [20]:
###############
# health care
###############

query = (""" (SUBDIV_NAME LIKE '%HOSP%' Or SUBDIV_NAME LIKE '%CLINIC%' Or 
              SUBDIV_NAME LIKE '%MEDICAL%') OR (PARCEL_ID IN ('076590005','076590006','076590007','070530072',
                '070530069','070530036','070530054')) """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# add parcels that overlap with healthcare points - currently excluding nursing homes
healthcare_facilties_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\HealthCareFacilities.shp','healthcare_facilties_lyr') 
query = """ TYPE NOT IN ('NURSING HOME', 'ASSISTED LIVING FACILITY') """
arcpy.SelectLayerByAttribute_management(healthcare_facilties_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=healthcare_facilties_lyr,
                                       selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

46 "healthcare" parcels were selected.
80401 parcels remain...


In [21]:
###############
# government
###############

query = (""" TAXEXEMPT_TYPE IN ('100-Federal Government','110-USA','200-State of Utah','400-Weber County', 
            '450-Weber County Library','501-Ogden City','502-South Ogden City','503-Ogden City Redevelopment','505-Roy City','506-Marriott-Slaterville',
            '507-North Ogden City','508-Farr West City','509-Pleasant View City','510-Plain City','511-Harrisville City',
            '512-Riverdale City','513-Uintah (City of)','514-West Haven City','515-Wash Terrace City','516-Hooper City',
            '517-South Weber (City of)','800-Charitable Organization','010210067') """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

3763 "government" parcels were selected.
76638 parcels remain...


In [22]:
###############
# industrial
###############

query = (""" PROP_CLASS IN ('200-INDUSTRIAL') or PARCEL_ID IN ('070820020','070820021','070820028','070820049',
                                                               '070820025','070820019','051260032','051250010',
                                                               '051250009','051250068','051250067','040490027',
                                                               '040470006','040480026','040470005','040430023',
                                                               '040480028','040480024','051250073','040430007',
                                                               '040430008','040490032','040470012','040430022',
                                                               '051250053','051250046','040430012','040460047',
                                                               '040490007','040430051','040470007','040430014',
                                                               '051250047','051250049','040480023','040430011',
                                                               '040460031','040430021','040430029','040430050',
                                                               '070520061','070520062','070520059','070520058',
                                                               '070520055','030080018','030360007','121270002',
                                                               '121270003','121240027','121240022','121250050',
                                                               '114340001','170660038','170660013','190410029',
                                                               '190410028','190410026','190640006','190640004',
                                                               '190640003','110200072','110200077','110200076',
                                                               '114090009','114090008','110200040','114090007',
                                                               '121250050')""")
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="3",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

857 "industrial" parcels were selected.
75781 parcels remain...


In [23]:
###############
# commercial/retail
###############

query = (""" PROP_CLASS IN ('500- COMMERCIAL') or PROP_CLASS IN ('952- EXEMPT - COM') OR PARCEL_ID IN ('051410006',
             '051410001','051410019','051380025','051380024','051380023','051380026','051380027','051380030',
             '051380029','051380028','051400008','051400020','051400004','051400010','051400014','051400018',
             '051390077','051250022','051210030','051250001','051240007','121080015','121080019','030450002',
             '051390078','051390090','051390086','051390081','051390064','051960003','060300045','070520059',
             '040260013','040260015','040260041','040290030','040290029','040410063','040410003','113520001') """)

tag="commercial_retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
comm_retail = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2346 "commercial_retail" parcels were selected.
73435 parcels remain...


In [24]:
#################
# roads/corridors
#################

query=""" (TAXEXEMPT_TYPE IN ('210-State Road Comm','402-Weber Pathways','611-Private Road','900-Railroad Properties')) OR 
          (PARCEL_ID IN ('170650033')) """
tag="roads"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1427 "roads" parcels were selected.
72008 parcels remain...


In [25]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

#=====================================================
# get parcels that overlap with reviewed common spaces
#======================================================

# open
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=ca_open_space,
                                       selection_type='ADD_TO_SELECTION')

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
                                       select_features=ca_open_space,
                                       selection_type='ADD_TO_SELECTION')

# query parcel attributes
query= (""" TAXEXEMPT_TYPE IN ('961-Cemetery') OR PARCEL_ID IN ('010580051','052090002','052090004','050270021',
                                                                '060600001','061680004','060820001','060820003',
                                                                '132030007','131130027','131140017','130870003',
                                                                '131140018','131130015','132030014','131130013',
                                                                '131130021','131130001','130800003','130870003',
                                                                '132030013','131130017','130730014','130720039',
                                                                '130720032','130720033','130720019','130390037',
                                                                '130390019','130720068','130390036','130720068') """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

202 "open_space" parcels were selected.
71806 parcels remain...


In [26]:
#################
# Undevelopable
#################

tag = "undevelopable"

query= (""" PARCEL_ID IN ('010520014', '010520007', '010520004', '010520008', '010520006','010520005', 
            '171410001','154590007') """)

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2 "undevelopable" parcels were selected.
71804 parcels remain...


In [27]:
####################
# Drop small parcels
####################

query = """ Shape_Area < 375 """

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4032 "undevelopable" parcels were selected.
67772 parcels remain...


In [28]:
#################
# Vacant land
#################

# query= (""" PROP_CLASS IN ('901- VAC - RES - DEV','902- C/I VACANT','911- RES LOT', '917- REC-VAC-LOT', '922- PUD LOT',
#             '999- UNDEVELOPED VL') """)
tag = "vacant"

# select the features
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
#                                        select_features=ca_vacant,
#                                        selection_type='ADD_TO_SELECTION')

# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
#                                        select_features=ca_vacant,
#                                        selection_type='ADD_TO_SELECTION')

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

67772 "vacant" parcels were selected.
0 parcels remain...


In [29]:
# # export remaining parcels
# arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')

In [30]:
##################################
# Merge the parcels back together
##################################

arcpy.env.workspace = gdb
featureclasses = arcpy.ListFeatureClasses()
merged_parcels = arcpy.Merge_management(featureclasses, os.path.join(scratch, 'merged_parcels'),
                                        add_source='ADD_SOURCE_INFO')
merged_parcels_lyr = arcpy.MakeFeatureLayer_management(merged_parcels, 'merged_parcels_lyr') 


## Use the Generalized Future Land Use layer to set the parcel zoning

In [31]:
# set land use type on parcels using glfu
gflu = '.\Inputs\FutureLandUse2020.gdb\FutureLandUse2020'
gflu_lyr = arcpy.MakeFeatureLayer_management(gflu, 'gflu_lyr') 


##################
# Type 1 (SF)
##################

# query for land use type
query = """ GenLUType IN ('Any Development','Any Residential', 'Mixed Use SF', 'Residential SF', 'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'t'".format(tag),
                                expression_type="PYTHON3")


##################
# Type 2 (MF)
##################

# query for land use type
query = """ GenLUType IN ('Any Residential', 'Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 
                          'Residential MF/Office', 'Residential MF', 'Mixed Use MF', 'Mixed Use SF', 'Residential/Office', 
                          'Residential/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 3 (Industrial)
#######################

# query for land use type
query = """ GenLUType IN ('Industrial', 'Any Development', 'Industrial/Mixed Use MF') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 4 (Retail)
#######################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'t'".format(tag),
                                expression_type="PYTHON3")


#######################
# Type 5 (Office)
#######################

# query for land use type
query = """ GenLUType IN ('Retail/Office', 'Office', 'Any Commercial', 'Residential/Office', 'Residential MF/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 6 (Government and Education)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Government/Education') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'t'".format(tag),
                                expression_type="PYTHON3")


####################################
# Type 7 (Mixed Use)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 'Mixed Use MF', 'Mixed Use SF', 
                          'Residential MF/Office', 'Residential SF/Retail', 'Residential/Office', 'Residential/Retail', 
                          'Retail/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 8 (Other)
###################################

# # query for land use type
# query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
#                           'Residential SF/Retail') """
# arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# # select the parcels
# arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
#                                        select_features=gflu_lyr,
#                                        selection_type='NEW_SELECTION')

# arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'t'".format(tag),
#                                 expression_type="PYTHON3")

#################################
# Undevelopable
#################################

# query for land use type
query = """ GenLUType IN ('NoBuild') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr, overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

query = """ NoBuild = 1 """
arcpy.SelectLayerByAttribute_management(merged_parcels_lyr, 'NEW_SELECTION', query)


# set all types to false
arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'f'".format(tag),
                                expression_type="PYTHON3")

# clear selected parcels
arcpy.SelectLayerByAttribute_management(merged_parcels_lyr, 'CLEAR_SELECTION', query)

id,value
0,a Layer object
1,-1


## Eject buildings from parcel data

In [34]:
parcels_buildings_sdf = pd.DataFrame.spatial.from_featureclass(merged_parcels)

#######################
# Fix the unit counts
#######################

# convert unit count columns to int
parcels_buildings_sdf.loc[(parcels_buildings_sdf['HOUSE_CNT'].isnull() == True ), 'HOUSE_CNT'] = 0
# parcels_buildings_sdf['HOUSE_CNT'] = parcels_buildings_sdf['HOUSE_CNT'].astype(int)
parcels_buildings_sdf.loc[(parcels_buildings_sdf['ap_count'].isnull() == True ), 'ap_count'] = 0
parcels_buildings_sdf['ap_count'] = parcels_buildings_sdf['ap_count'].astype(int)


# create new count field and calculate
parcels_buildings_sdf['UNIT_COUNT'] = parcels_buildings_sdf['ap_count']

# fix single family (non-pud)
parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['SUBTYPE_WFRC'] == 'single_family'), 
                 'UNIT_COUNT'] = 1

# fix duplex
parcels_buildings_sdf.loc[(parcels_buildings_sdf['SUBTYPE_WFRC'] == 'duplex'), 'UNIT_COUNT'] = 2

# # fix triplex-quadplex
# parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] < parcels_buildings_sdf['HOUSE_CNT']) & 
#                  (parcels_buildings_sdf['SUBTYPE_WFRC'] == 'triplex-quadplex'), 
#                  'UNIT_COUNT'] = parcels_buildings_sdf['HOUSE_CNT']


###############################
# Subset to necessary columns
###############################

parcels_buildings_sdf = parcels_buildings_sdf[['COUNTY_NAME','COUNTY_ID','PARCEL_ID','TOTAL_MKT_VALUE',
                                               'LAND_MKT_VALUE','PARCEL_ACRES','UNIT_COUNT','BLDG_SQFT',
                                               'FLOORS_CNT','BUILT_YR','EFFBUILT_YR','max_dua','max_far','max_height', 
                                               'type1','type2','type3','type4','type5','type6','type7','type8', 
                                               'agriculture','basebldg','NoBuild','redev_friction','building_type_id',
                                               'SHAPE']].copy()



#############################################
# won't need these till all counties are done
#############################################

# create new REMM parcel IDs
parcels_buildings_sdf['parcel_id_remm'] = np.arange(len(parcels_buildings_sdf))

# copy table and only keep rows if basebldg = 1
buildings_df = parcels_buildings_sdf[parcels_buildings_sdf['basebldg'] == 1].copy()
buildings_df = buildings_df[['parcel_id_remm','BLDG_SQFT','TOTAL_MKT_VALUE','UNIT_COUNT',
                             'building_type_id', 'FLOORS_CNT','BUILT_YR']].copy()

# generate building id
buildings_df['building_id'] = np.arange(len(buildings_df))

# add new attributes
buildings_df['non_residential_sqft'] = 0
buildings_df['note'] = 'base'
buildings_df['residential_units'] = 0
buildings_df['unit_price_non_residential'] = 0
buildings_df['job_spaces'] = 0

buildings_df.loc[buildings_df['building_type_id'].isin([1,2])== False, 'non_residential_sqft'] = buildings_df['BLDG_SQFT']
buildings_df.loc[buildings_df['building_type_id'].isin([1,2])== True, 'residential_units'] = buildings_df['UNIT_COUNT']

# rename building columns
buildings_df.rename(columns={"BLDG_SQFT": "bldg_sqft", "FLOORS_CNT": "stories",'BUILT_YR':'year_built'})

# export
parcels_buildings_sdf.spatial.to_featureclass(location= os.path.join(final, 'davis_parcels'),sanitize_columns=False)
parcels_buildings_sdf.to_csv(os.path.join(outputs, 'weber_parcels.csv'), index=False)
buildings_df.to_csv(os.path.join(outputs, 'weber_buildings.csv'), index=False)